In [ ]:
!pip install -q --upgrade fastcore fastai

In [ ]:
import fastai
print(fastai.__version__ )

2.6.3


In [ ]:
from fastai.tabular.all import *

In [ ]:
import numpy as np
import pandas as pd 
import lightgbm as lgb
import gc
import math
from fastai.tabular import *
# from fastai.tabular.transform import add_cyclic_datepart
from sklearn import metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold, StratifiedKFold
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import datetime
from tqdm import tqdm
from tqdm import tqdm_notebook

In [ ]:
def mae(y_true, y_pred):
    return mae(y_true, y_pred)

**Loading datasets from my drive using gdown:**

In [ ]:
!gdown https://drive.google.com/uc?id=1xTlto7tbQijeajjwBMFpBso-jfRsbHya
!unzip /content/layerai-air-quality-prediction-challenge.zip

**Reading the datasets:**

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub  = pd.read_csv('SampleSubmission.csv')

In [ ]:
train.shape, test.shape, sub.shape

((9923, 71), (4254, 70), (4254, 2))

In [ ]:
train.head()

ID        date    device  site_latitude  site_longitude  \
0  ID_JBYXJEUZ8X  2020-03-18  ANQ16PZJ       0.353465       32.560322   
1  ID_PEJJNLBUPR  2019-10-13  AB6051M4       0.390741       32.582257   
2  ID_21L8UZQO1T  2019-12-21  A0WN66FH       0.285751       32.578325   
3  ID_GNW2G8J4T8  2020-02-20     aq_41      -1.244985       29.989236   
4  ID_4XOX2V6SK0  2019-04-17     aq_41      -1.244985       29.989236   

   humidity  temp_mean  SulphurDioxide_SO2_column_number_density  \
0  0.846844  22.038542                                 -0.000138   
1  0.874493  21.545833                                       NaN   
2  0.729340  23.568655                                       NaN   
3  0.919271  16.338542                                 -0.000111   
4  0.858365  16.602083                                 -0.000127   

   SulphurDioxide_SO2_column_number_density_amf  \
0                                      1.042629   
1                                           NaN   
2                                           NaN   
3                                      0.750228   
4                                      0.792634   

   SulphurDioxide_SO2_slant_column_number_density  ...  \
0                                       -0.000143  ...   
1                                             NaN  ...   
2                                             NaN  ...   
3                                       -0.000083  ...   
4                                       -0.000101  ...   

   Cloud_cloud_top_height  Cloud_cloud_base_pressure  Cloud_cloud_base_height  \
0             4181.792969               69589.453125              3181.793457   
1                     NaN                        NaN                      NaN   
2             4440.391113               67341.109375              3440.390869   
3                     NaN                        NaN                      NaN   
4             4842.057488               64347.788635              3842.057585   

   Cloud_cloud_optical_depth  Cloud_surface_albedo  \
0                   7.799846              0.278111   
1                        NaN                   NaN   
2                  62.496590              0.416631   
3                        NaN                   NaN   
4                   4.971117              0.277157   

   Cloud_sensor_azimuth_angle  Cloud_sensor_zenith_angle  \
0                   76.618553                   9.970293   
1                         NaN                        NaN   
2                  -97.300758                  63.690605   
3                         NaN                        NaN   
4                   74.786025                  32.583070   

   Cloud_solar_azimuth_angle  Cloud_solar_zenith_angle      pm2_5  
0                 -93.110069                 19.214193  63.853333  
1                        NaN                       NaN  32.507083  
2                -127.856804                 40.857922  80.290833  
3                        NaN                       NaN  42.601687  
4                 -57.273638                 21.864529  89.642229  

[5 rows x 71 columns]

In [ ]:
test.head()

ID        date    device  site_latitude  site_longitude  \
0  ID_UOH62J0XHX  2020-10-02     aq_43       0.289040       32.589580   
1  ID_4OPWSB0UHJ  2020-12-07     aq_54       0.356400       32.573000   
2  ID_3SLMNNG1Z3  2020-11-16     aq_59       0.381576       32.647109   
3  ID_MFUHTXYPM4  2020-09-05  AW66FF7V       0.379562       32.553714   
4  ID_TY1DAND8ZP  2020-12-15     aq_45       0.344000       32.553000   

   humidity  temp_mean  SulphurDioxide_SO2_column_number_density  \
0  0.756840  23.849653                                  0.000124   
1  0.810854  22.075347                                       NaN   
2  0.883253  20.050694                                       NaN   
3  0.782583  22.248958                                  0.000176   
4  0.770116  22.565217                                       NaN   

   SulphurDioxide_SO2_column_number_density_amf  \
0                                      0.839054   
1                                           NaN   
2                                           NaN   
3                                      0.730853   
4                                           NaN   

   SulphurDioxide_SO2_slant_column_number_density  ...  \
0                                        0.000104  ...   
1                                             NaN  ...   
2                                             NaN  ...   
3                                        0.000129  ...   
4                                             NaN  ...   

   Cloud_cloud_top_pressure  Cloud_cloud_top_height  \
0              36716.141728             8325.718024   
1              63086.023438             4060.879639   
2              46783.820820             6456.160528   
3              45528.261719             6682.570801   
4              65512.797785             3757.637022   

   Cloud_cloud_base_pressure  Cloud_cloud_base_height  \
0               41969.524699              7325.718482   
1               70871.984375              3060.879639   
2               53084.224103              5456.160528   
3               51503.203125              5682.570801   
4               73461.827324              2757.637022   

   Cloud_cloud_optical_depth  Cloud_surface_albedo  \
0                   2.632129              0.210430   
1                   2.773685              0.344436   
2                   3.862119              0.326882   
3                   1.937542              0.212181   
4                   3.733908              0.262021   

   Cloud_sensor_azimuth_angle  Cloud_sensor_zenith_angle  \
0                   74.431975                  35.449770   
1                  -97.300392                  63.489811   
2                  -97.699519                  58.824120   
3                   75.361595                  24.835907   
4                   76.537080                  11.026513   

   Cloud_solar_azimuth_angle  Cloud_solar_zenith_angle  
0                -101.546713                 21.006060  
1                -125.937134                 41.709042  
2                -120.683819                 40.183680  
3                 -72.224319                 20.675266  
4                -139.072620                 32.046360  

[5 rows x 70 columns]

In [ ]:
train.isnull().sum()

ID                               0
date                             0
device                           0
site_latitude                    0
site_longitude                   0
                              ... 
Cloud_sensor_azimuth_angle    1509
Cloud_sensor_zenith_angle     1509
Cloud_solar_azimuth_angle     1509
Cloud_solar_zenith_angle      1509
pm2_5                            0
Length: 71, dtype: int64

In [ ]:
test.isnull().sum()

ID                              0
date                            0
device                          0
site_latitude                   0
site_longitude                  0
                             ... 
Cloud_surface_albedo          515
Cloud_sensor_azimuth_angle    515
Cloud_sensor_zenith_angle     515
Cloud_solar_azimuth_angle     515
Cloud_solar_zenith_angle      515
Length: 70, dtype: int64

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9923 entries, 0 to 9922
Data columns (total 71 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   ID                                                        9923 non-null   object 
 1   date                                                      9923 non-null   object 
 2   device                                                    9923 non-null   object 
 3   site_latitude                                             9923 non-null   float64
 4   site_longitude                                            9923 non-null   float64
 5   humidity                                                  9923 non-null   float64
 6   temp_mean                                                 9903 non-null   float64
 7   SulphurDioxide_SO2_column_number_density                  4291 non-null   float64
 8   SulphurDioxide_SO2

In [ ]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x=="nan":
        return np.nan
    else :
        return float(x)

In [ ]:
features=["humidity","temp_mean"]

def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [ ]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    x["p1_"+col_name]=x[col_name].apply(percentile(1))
    x["p5_"+col_name]=x[col_name].apply(percentile(5))
    x["p10_"+col_name]=x[col_name].apply(percentile(10))
    x["p20_"+col_name]=x[col_name].apply(percentile(20))
    x["p30_"+col_name]=x[col_name].apply(percentile(30))
    x["p40_"+col_name]=x[col_name].apply(percentile(40))
    x["p60_"+col_name]=x[col_name].apply(percentile(60))
    x["p70_"+col_name]=x[col_name].apply(percentile(70))
    x["p80_"+col_name]=x[col_name].apply(percentile(80))
    x["p90_"+col_name]=x[col_name].apply(percentile(90))
    x["p95_"+col_name]=x[col_name].apply(percentile(95))
    x["p99_"+col_name]=x[col_name].apply(percentile(99))
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]

In [ ]:
for col_name in tqdm(features):
    train=aggregate_features(train,col_name)
    test=aggregate_features(test,col_name)

100%|██████████| 2/2 [00:28<00:00, 14.21s/it]


In [ ]:
train[['temp_mean']].isnull().sum()

temp_mean    20
dtype: int64

In [ ]:
cols_to_fillna=["temp_mean"]                  

for i in cols_to_fillna:
    train[i].fillna(np.mean(train[i]),inplace=True)
    test[i].fillna(np.mean(test[i]),inplace=True)

In [ ]:
train.head()

ID        date    device  site_latitude  site_longitude  \
0  ID_JBYXJEUZ8X  2020-03-18  ANQ16PZJ       0.353465       32.560322   
1  ID_PEJJNLBUPR  2019-10-13  AB6051M4       0.390741       32.582257   
2  ID_21L8UZQO1T  2019-12-21  A0WN66FH       0.285751       32.578325   
3  ID_GNW2G8J4T8  2020-02-20     aq_41      -1.244985       29.989236   
4  ID_4XOX2V6SK0  2019-04-17     aq_41      -1.244985       29.989236   

   humidity  temp_mean  SulphurDioxide_SO2_column_number_density  \
0  0.846844  22.038542                                 -0.000138   
1  0.874493  21.545833                                       NaN   
2  0.729340  23.568655                                       NaN   
3  0.919271  16.338542                                 -0.000111   
4  0.858365  16.602083                                 -0.000127   

   SulphurDioxide_SO2_column_number_density_amf  \
0                                      1.042629   
1                                           NaN   
2                                           NaN   
3                                      0.750228   
4                                      0.792634   

   SulphurDioxide_SO2_slant_column_number_density  ...  p10_temp_mean  \
0                                       -0.000143  ...      22.038542   
1                                             NaN  ...      21.545833   
2                                             NaN  ...      23.568655   
3                                       -0.000083  ...      16.338542   
4                                       -0.000101  ...      16.602083   

   p20_temp_mean  p30_temp_mean  p40_temp_mean  p60_temp_mean  p70_temp_mean  \
0      22.038542      22.038542      22.038542      22.038542      22.038542   
1      21.545833      21.545833      21.545833      21.545833      21.545833   
2      23.568655      23.568655      23.568655      23.568655      23.568655   
3      16.338542      16.338542      16.338542      16.338542      16.338542   
4      16.602083      16.602083      16.602083      16.602083      16.602083   

   p80_temp_mean  p90_temp_mean  p95_temp_mean  p99_temp_mean  
0      22.038542      22.038542      22.038542      22.038542  
1      21.545833      21.545833      21.545833      21.545833  
2      23.568655      23.568655      23.568655      23.568655  
3      16.338542      16.338542      16.338542      16.338542  
4      16.602083      16.602083      16.602083      16.602083  

[5 rows x 109 columns]

**Converting to datetime to use date features.**

In [ ]:
train[['date']].head()

date
0  2020-03-18
1  2019-10-13
2  2019-12-21
3  2020-02-20
4  2019-04-17

In [ ]:
train['date'] =  pd.to_datetime(train['date'], format='%Y-%m-%d')
test['date'] =  pd.to_datetime(test['date'], format='%Y-%m-%d')

**Center of the city**

In [ ]:
from geopy.geocoders import Nominatim
import time
from pprint import pprint

app = Nominatim(user_agent="tutorial")
# get location raw data
location = app.geocode("Kampala, Uganda").raw
# print raw data
pprint(location)

{'boundingbox': ['0.2143567', '0.4057676', '32.5098753', '32.6687413'],
 'class': 'boundary',
 'display_name': 'Kampala Capital City, Kampala, Central Region, Uganda',
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png',
 'importance': 0.7925411725065692,
 'lat': '0.3177137',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
            'https://osm.org/copyright',
 'lon': '32.5813539',
 'osm_id': 10546821,
 'osm_type': 'relation',
 'place_id': 285843136,
 'type': 'administrative'}


In [ ]:
def add_center_feature(df):
    site_latitude = float(location['lat'])
    site_longitude = float(location['lon'])
    df['center_lat'] = [site_latitude] * df.shape[0]
    df['center_long'] = [site_longitude] * df.shape[0]
    return df

add_center_feature(train);
add_center_feature(test);

In [ ]:
ID_COL, TARGET_COL = 'ID', 'pm2_5'
df = pd.concat([train, test]).reset_index(drop=True)
features = [c for c in df.columns if c not in ['ID', 'date', 'pm2_5']]

**Checking out the City Location:**

In [ ]:
!pip install astral==2.2 --quiet

In [ ]:
import astral
from astral.sun import zenith_and_azimuth, elevation
from astral.geocoder import database, lookup
city = lookup("Kampala", database())
city

LocationInfo(name='Kampala', region='Uganda', timezone='Africa/Kampala', latitude=0.3333333333333333, longitude=32.5)

#Time Series Features
A. Using Previous and Future Target Values.
Pollution levels yesterday could tell about the pollution levels today, similary pollution levels tomorrow, could also tell about pollution levels today. We add this information to our dataset, using shift and expanding functions


In [ ]:
for i in range(1, 20):
    df[f'prev_target_{i}'] = df.sort_values(by='date')[TARGET_COL].fillna(method='ffill').shift(i).sort_index()
    df[f'next_target_{i}'] = df.sort_values(by='date')[TARGET_COL].fillna(method='bfill').shift(-i).sort_index()

for i in tqdm_notebook(range(1, 15)):
    df[f'magic_{i}'] = df.sort_values(by='date')[TARGET_COL].shift(i).expanding().mean().fillna(method='ffill').sort_index()
    df[f'magic2_{i}'] = df.sort_values(by='date')[TARGET_COL].shift(-i).expanding().mean().fillna(method='bfill').sort_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/14 [00:00<?, ?it/s]

#B. Cyclic Components
Date has cyclic components, for example monday repeats after every 7 days, adding this as a feature can help capture more information from the dataset.

To read more visit:

https://docs.fast.ai/tabular.transform.html#add_cyclic_datepart


In [ ]:
# add_datepart(df_raw, 'saledate', drop=False)

In [ ]:
df['date'] = df['date'].astype('datetime64')

# # use sin and cos to capture cyclic pattern
diff = df['date'] - min(df['date'])

# # yearly pattern
days = diff.dt.days
df['year_1_sin'] = np.sin(2 * math.pi * days / (365 * 1)) 
df['year_1_cos'] = np.cos(2 * math.pi * days / (365 * 1))
df['year_2_sin'] = np.sin(2 * math.pi * days / (365 * 2)) 
df['year_2_cos'] = np.cos(2 * math.pi * days / (365 * 2)) 
df['year_3_sin'] = np.sin(2 * math.pi * days / (365 * 3)) 
df['year_3_cos'] = np.cos(2 * math.pi * days / (365 * 3)) 
df['year_4_sin'] = np.sin(2 * math.pi * days / (365 * 4)) 
df['year_4_cos'] = np.cos(2 * math.pi * days / (365 * 4)) 

# # daily pattern
seconds = diff.dt.seconds
df['day_1_sin'] = np.sin(2 * math.pi * seconds / ( 3600 * 24 * 1))
df['day_1_cos'] = np.cos(2 * math.pi * seconds / ( 3600 * 24 * 1))
df['day_2_sin'] = np.sin(2 * math.pi * seconds / ( 3600 * 24 * 2))
df['day_2_cos'] = np.cos(2 * math.pi * seconds / ( 3600 * 24 * 2))
df['day_7_sin'] = np.sin(2 * math.pi * seconds / ( 3600 * 24 * 7))
df['day_7_cos'] = np.cos(2 * math.pi * seconds / ( 3600 * 24 * 7))    

#Manhattan distnace


In [ ]:
def manhattan_distance(lat,lon):
    a = np.abs(lat -lon)
    return a
df['manhattan_dist'] = manhattan_distance(df['site_latitude'].values, df['site_longitude'].values,)

#Bearing


In [ ]:
def bearing_array(lat, lng):
    AVG_EARTH_RADIUS = 6371  # in km
    lng_delta_rad = np.radians(lat - lng)
    lat, lng = map(np.radians, (lat, lng))
    y = np.sin(lng_delta_rad) * np.cos(lat)
    x = np.cos(lat) * np.sin(lat) - np.sin(lng) * np.cos(lng) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))
df['bearing'] = bearing_array(df['site_latitude'].values, df['site_longitude'].values,)
df.head()

ID       date    device  site_latitude  site_longitude  \
0  ID_JBYXJEUZ8X 2020-03-18  ANQ16PZJ       0.353465       32.560322   
1  ID_PEJJNLBUPR 2019-10-13  AB6051M4       0.390741       32.582257   
2  ID_21L8UZQO1T 2019-12-21  A0WN66FH       0.285751       32.578325   
3  ID_GNW2G8J4T8 2020-02-20     aq_41      -1.244985       29.989236   
4  ID_4XOX2V6SK0 2019-04-17     aq_41      -1.244985       29.989236   

   humidity  temp_mean  SulphurDioxide_SO2_column_number_density  \
0  0.846844  22.038542                                 -0.000138   
1  0.874493  21.545833                                       NaN   
2  0.729340  23.568655                                       NaN   
3  0.919271  16.338542                                 -0.000111   
4  0.858365  16.602083                                 -0.000127   

   SulphurDioxide_SO2_column_number_density_amf  \
0                                      1.042629   
1                                           NaN   
2                                           NaN   
3                                      0.750228   
4                                      0.792634   

   SulphurDioxide_SO2_slant_column_number_density  ...  year_4_sin  \
0                                       -0.000143  ...    0.999769   
1                                             NaN  ...    0.793572   
2                                             NaN  ...    0.936881   
3                                       -0.000083  ...    0.995521   
4                                       -0.000101  ...    0.145799   

   year_4_cos  day_1_sin  day_1_cos  day_2_sin  day_2_cos  day_7_sin  \
0   -0.021516        0.0        1.0        0.0        1.0        0.0   
1    0.608477        0.0        1.0        0.0        1.0        0.0   
2    0.349647        0.0        1.0        0.0        1.0        0.0   
3    0.094537        0.0        1.0        0.0        1.0        0.0   
4    0.989314        0.0        1.0        0.0        1.0        0.0   

   day_7_cos  manhattan_dist     bearing  
0        1.0       32.206858 -125.319438  
1        1.0       32.191516 -125.298847  
2        1.0       32.292575 -125.321748  
3        1.0       31.234221 -127.087130  
4        1.0       31.234221 -127.087130  

[5 rows x 193 columns]

# Using simple features extracted from the date itself.
Like dayofweek, weekend or not, month, days_in_month, etc.

In [ ]:
for attr in ['day', 'year', 'hour', 'month', 'week', 'dayofweek', 'weekofyear', 'days_in_month', 'is_month_start', 'is_month_end', 'dayofyear']:
    df[attr] = getattr(df['date'].dt, attr)
df['is_weekend'] = (df['dayofweek'] >= 5)*1
df['fortnight'] = df['day']%15
df['3h'] = (df['hour']/ 3).astype(int)* 3  # Creating 3hours time based features
df['which_fortnight'] = df['day']//15

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  


In [ ]:
df = df.drop(['date'], axis=1)

**Adding Solar azimuth, zenith and elevation features:**

In [ ]:
def add_zenith_azimuth_elevation_feature(df):
    import datetime
    zen = []
    azi = []                      
    ele = []
    for i in tqdm(range(df.shape[0])):
        ze, az =  zenith_and_azimuth(city.observer, datetime.datetime(df.year[i], df.month[i], df.day[i], df.hour[i]))
        el = elevation(city.observer, datetime.datetime(df.year[i], df.month[i], df.day[i], df.hour[i]))
        zen.append(ze)
        azi.append(az)
        ele.append(el)
    df['elevation'] = ele
    df['azimuth'] = azi
    df['zenith'] = zen
    return df

add_zenith_azimuth_elevation_feature(df);

100%|██████████| 14177/14177 [00:01<00:00, 9529.06it/s]


**Adding holiday features:**

In [ ]:
!pip install holidays --quiet
import holidays
from datetime import date

In [ ]:
def add_holiday_feature(df):
  ZA_holidays = holidays.ZA()
  list_of_holidays = [int(date(df.year[i], df.month[i], df.day[i]) in ZA_holidays) for i in range(df.shape[0])]
  df['national_holiday'] = list_of_holidays
  return df

add_holiday_feature(df)

ID    device  site_latitude  site_longitude  humidity  \
0      ID_JBYXJEUZ8X  ANQ16PZJ       0.353465       32.560322  0.846844   
1      ID_PEJJNLBUPR  AB6051M4       0.390741       32.582257  0.874493   
2      ID_21L8UZQO1T  A0WN66FH       0.285751       32.578325  0.729340   
3      ID_GNW2G8J4T8     aq_41      -1.244985       29.989236  0.919271   
4      ID_4XOX2V6SK0     aq_41      -1.244985       29.989236  0.858365   
...              ...       ...            ...             ...       ...   
14172  ID_NYKGOYFBKZ     aq_76       0.357987       32.584100  0.786575   
14173  ID_IWO4EVLHSQ  AB6051M4       0.390741       32.582257  0.828441   
14174  ID_G1M1IMY9WU  AB6051M4       0.390741       32.582257  0.859490   
14175  ID_DXV65Q1QXI     aq_61       0.373078       32.628226  0.833803   
14176  ID_RY4ONQKO9E     aq_30       0.059604       32.460320  0.783191   

       temp_mean  SulphurDioxide_SO2_column_number_density  \
0      22.038542                                 -0.000138   
1      21.545833                                       NaN   
2      23.568655                                       NaN   
3      16.338542                                 -0.000111   
4      16.602083                                 -0.000127   
...          ...                                       ...   
14172  23.790833                                       NaN   
14173  21.801389                                       NaN   
14174  21.851042                                       NaN   
14175  21.948864                                       NaN   
14176  22.004514                                  0.000073   

       SulphurDioxide_SO2_column_number_density_amf  \
0                                          1.042629   
1                                               NaN   
2                                               NaN   
3                                          0.750228   
4                                          0.792634   
...                                             ...   
14172                                           NaN   
14173                                           NaN   
14174                                           NaN   
14175                                           NaN   
14176                                      0.745486   

       SulphurDioxide_SO2_slant_column_number_density  \
0                                           -0.000143   
1                                                 NaN   
2                                                 NaN   
3                                           -0.000083   
4                                           -0.000101   
...                                               ...   
14172                                             NaN   
14173                                             NaN   
14174                                             NaN   
14175                                             NaN   
14176                                        0.000054   

       SulphurDioxide_cloud_fraction  ...  is_month_end  dayofyear  \
0                           0.291358  ...         False         78   
1                                NaN  ...         False        286   
2                                NaN  ...         False        355   
3                           0.000000  ...         False         51   
4                           0.087338  ...         False        107   
...                              ...  ...           ...        ...   
14172                            NaN  ...         False        238   
14173                            NaN  ...         False        234   
14174                            NaN  ...         False        312   
14175                            NaN  ...         False        254   
14176                       0.215744  ...         False        277   

       is_weekend  fortnight  3h  which_fortnight  elevation     azimuth  \
0               0          3   0                1 -59.503627   91.118398   
1               1         13   0 

**Separating Train and Test sets:**

In [ ]:
train = df[:train.shape[0]].reset_index(drop=True)
test = df[train.shape[0]:].reset_index(drop=True)

target = train[TARGET_COL]

In [ ]:
train.shape, test.shape

((9923, 211), (4254, 211))

In [ ]:
train.head()

ID    device  site_latitude  site_longitude  humidity  \
0  ID_JBYXJEUZ8X  ANQ16PZJ       0.353465       32.560322  0.846844   
1  ID_PEJJNLBUPR  AB6051M4       0.390741       32.582257  0.874493   
2  ID_21L8UZQO1T  A0WN66FH       0.285751       32.578325  0.729340   
3  ID_GNW2G8J4T8     aq_41      -1.244985       29.989236  0.919271   
4  ID_4XOX2V6SK0     aq_41      -1.244985       29.989236  0.858365   

   temp_mean  SulphurDioxide_SO2_column_number_density  \
0  22.038542                                 -0.000138   
1  21.545833                                       NaN   
2  23.568655                                       NaN   
3  16.338542                                 -0.000111   
4  16.602083                                 -0.000127   

   SulphurDioxide_SO2_column_number_density_amf  \
0                                      1.042629   
1                                           NaN   
2                                           NaN   
3                                      0.750228   
4                                      0.792634   

   SulphurDioxide_SO2_slant_column_number_density  \
0                                       -0.000143   
1                                             NaN   
2                                             NaN   
3                                       -0.000083   
4                                       -0.000101   

   SulphurDioxide_cloud_fraction  ...  is_month_end  dayofyear  is_weekend  \
0                       0.291358  ...         False         78           0   
1                            NaN  ...         False        286           1   
2                            NaN  ...         False        355           1   
3                       0.000000  ...         False         51           0   
4                       0.087338  ...         False        107           0   

   fortnight  3h  which_fortnight  elevation     azimuth      zenith  \
0          3   0                1 -59.503627   91.118398  149.503627   
1         13   0                0 -53.487793  102.346188  143.487793   
2          6   0                1 -50.452504  128.137142  140.452504   
3          5   0                1 -59.173639  111.651395  149.173639   
4          2   0                1 -55.906514   70.885679  145.906514   

   national_holiday  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  

[5 rows x 211 columns]

In [ ]:
test.head()

ID    device  site_latitude  site_longitude  humidity  \
0  ID_UOH62J0XHX     aq_43       0.289040       32.589580  0.756840   
1  ID_4OPWSB0UHJ     aq_54       0.356400       32.573000  0.810854   
2  ID_3SLMNNG1Z3     aq_59       0.381576       32.647109  0.883253   
3  ID_MFUHTXYPM4  AW66FF7V       0.379562       32.553714  0.782583   
4  ID_TY1DAND8ZP     aq_45       0.344000       32.553000  0.770116   

   temp_mean  SulphurDioxide_SO2_column_number_density  \
0  23.849653                                  0.000124   
1  22.075347                                       NaN   
2  20.050694                                       NaN   
3  22.248958                                  0.000176   
4  22.565217                                       NaN   

   SulphurDioxide_SO2_column_number_density_amf  \
0                                      0.839054   
1                                           NaN   
2                                           NaN   
3                                      0.730853   
4                                           NaN   

   SulphurDioxide_SO2_slant_column_number_density  \
0                                        0.000104   
1                                             NaN   
2                                             NaN   
3                                        0.000129   
4                                             NaN   

   SulphurDioxide_cloud_fraction  ...  is_month_end  dayofyear  is_weekend  \
0                       0.063487  ...         False        276           0   
1                            NaN  ...         False        342           0   
2                            NaN  ...         False        321           0   
3                       0.085770  ...         False        249           1   
4                            NaN  ...         False        350           0   

   fortnight  3h  which_fortnight  elevation     azimuth      zenith  \
0          2   0                0 -54.705783   95.886742  144.705783   
1          7   0                0 -49.605237  125.943621  139.605237   
2          1   0                1 -49.872542  119.536970  139.872542   
3          5   0                0 -56.499584   77.247031  146.499584   
4          0   0                1 -50.024360  127.459473  140.024360   

   national_holiday  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  

[5 rows x 211 columns]

In [ ]:
numerical_feats= train.dtypes[train.dtypes != "object"].index; numerical_feats

Index(['site_latitude', 'site_longitude', 'humidity', 'temp_mean',
       'SulphurDioxide_SO2_column_number_density',
       'SulphurDioxide_SO2_column_number_density_amf',
       'SulphurDioxide_SO2_slant_column_number_density',
       'SulphurDioxide_cloud_fraction', 'SulphurDioxide_sensor_azimuth_angle',
       'SulphurDioxide_sensor_zenith_angle',
       ...
       'is_month_end', 'dayofyear', 'is_weekend', 'fortnight', '3h',
       'which_fortnight', 'elevation', 'azimuth', 'zenith',
       'national_holiday'],
      dtype='object', length=209)

In [ ]:
categorical_feats= train.dtypes[train.dtypes == "object"].index; categorical_feats

Index(['ID', 'device'], dtype='object')

In [ ]:
train_id = train['ID']
test_id = test['ID']
y = train['pm2_5']

In [ ]:
!pip install category_encoders --quiet

In [ ]:
import category_encoders as ce
encoder = ce.CatBoostEncoder(cols=['device'])
encoder.fit(train, y)
train = encoder.transform(train)
test = encoder.transform(test)

CatBoostEncoder(cols=['device'])

In [ ]:
train.shape,test.shape

((9923, 211), (4254, 211))

In [ ]:
train.head()

ID     device  site_latitude  site_longitude  humidity  \
0  ID_JBYXJEUZ8X  69.532179       0.353465       32.560322  0.846844   
1  ID_PEJJNLBUPR  60.356548       0.390741       32.582257  0.874493   
2  ID_21L8UZQO1T  59.221183       0.285751       32.578325  0.729340   
3  ID_GNW2G8J4T8  62.146021      -1.244985       29.989236  0.919271   
4  ID_4XOX2V6SK0  62.146021      -1.244985       29.989236  0.858365   

   temp_mean  SulphurDioxide_SO2_column_number_density  \
0  22.038542                                 -0.000138   
1  21.545833                                       NaN   
2  23.568655                                       NaN   
3  16.338542                                 -0.000111   
4  16.602083                                 -0.000127   

   SulphurDioxide_SO2_column_number_density_amf  \
0                                      1.042629   
1                                           NaN   
2                                           NaN   
3                                      0.750228   
4                                      0.792634   

   SulphurDioxide_SO2_slant_column_number_density  \
0                                       -0.000143   
1                                             NaN   
2                                             NaN   
3                                       -0.000083   
4                                       -0.000101   

   SulphurDioxide_cloud_fraction  ...  is_month_end  dayofyear  is_weekend  \
0                       0.291358  ...         False         78           0   
1                            NaN  ...         False        286           1   
2                            NaN  ...         False        355           1   
3                       0.000000  ...         False         51           0   
4                       0.087338  ...         False        107           0   

   fortnight  3h  which_fortnight  elevation     azimuth      zenith  \
0          3   0                1 -59.503627   91.118398  149.503627   
1         13   0                0 -53.487793  102.346188  143.487793   
2          6   0                1 -50.452504  128.137142  140.452504   
3          5   0                1 -59.173639  111.651395  149.173639   
4          2   0                1 -55.906514   70.885679  145.906514   

   national_holiday  
0                 0  
1                 0  
2                 0  
3                 0  
4                 0  

[5 rows x 211 columns]

#Get rid of "Highly Correlated Features":

In [ ]:
features = train.columns

num_feature = [col for col in features if col not in ['ID','pm2_5','date']] 
drop_columns=[]
corr = train[num_feature].corr()
# Drop highly correlated features 
columns = np.full((corr.shape[0],), True, dtype=bool)

for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >=0.999 :
            if columns[j]:
                columns[j] = False
                print('FEAT_A: {} FEAT_B: {} - Correlation: {}'.format(train[num_feature].columns[i] , train[num_feature].columns[j], corr.iloc[i,j]))
        elif corr.iloc[i,j] <= -0.995:
            if columns[j]:
                columns[j] = False

drop_columns = train[num_feature].columns[columns == False].values
print('drop_columns',len(drop_columns),drop_columns)

FEAT_A: site_longitude FEAT_B: bearing - Correlation: 0.9995057906894059
FEAT_A: humidity FEAT_B: max_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: min_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: mean_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: median_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p1_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p5_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p10_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p20_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p30_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p40_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p60_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p70_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p80_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p90_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p95_humidity - Correlation: 1.0
FEAT_A: humidity FEAT_B: p99_humidity - Correlation: 1.0
FEAT_A: temp_

In [ ]:
train.drop(drop_columns,inplace=True,axis =1 )
test.drop(drop_columns,inplace=True,axis =1 )

**Check and remove any constant features present inside the dataset**


In [ ]:
features2 = train.columns

cols = [col for col in features2 if col not in ['ID','pm2_5']]

# Check and remove constant features
columns_to_drop = [feat for feat in train[cols] if train[feat].std() ==0]
columns_to_drop

['std_humidity',
 'var_humidity',
 'ptp_humidity',
 'std_temp_mean',
 'var_temp_mean',
 'ptp_temp_mean',
 'center_long',
 'day_1_sin',
 'day_1_cos',
 'day_2_sin',
 'day_2_cos',
 'day_7_sin',
 'day_7_cos',
 'hour',
 '3h']

In [ ]:
train.drop(columns = columns_to_drop, inplace = True)
test.drop(columns = columns_to_drop, inplace = True)

In [ ]:
train.shape, test.shape

((9923, 109), (4254, 109))

In [ ]:
test = test.drop(['pm2_5'], axis = 1)

In [ ]:
X  = train.drop(['ID','pm2_5'], axis = 1)
y = train['pm2_5']

#MODEL: CatBoostRegressor

In [ ]:
!pip install catboost --quiet

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)  #15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    catf = np.where(X_train.dtypes != float)[0]
    m2  = CatBoostRegressor(n_estimators=10000,eval_metric='MAE',learning_rate=0.175, random_seed= 1234, use_best_model=True)
    m2.fit(X_train,y_train,cat_features=catf,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)        

0:	learn: 19.0162267	test: 18.9975308	test1: 17.9000976	best: 17.9000976 (0)	total: 101ms	remaining: 16m 50s
100:	learn: 7.3436738	test: 7.3181140	test1: 8.1370396	best: 8.1370396 (100)	total: 4.76s	remaining: 7m 47s
200:	learn: 6.2581924	test: 6.2381146	test1: 7.7311681	best: 7.7311655 (197)	total: 9.34s	remaining: 7m 35s
300:	learn: 5.5057739	test: 5.5052131	test1: 7.5955691	best: 7.5955691 (300)	total: 14s	remaining: 7m 29s
400:	learn: 4.8234352	test: 4.8359226	test1: 7.4739511	best: 7.4739511 (400)	total: 18.5s	remaining: 7m 23s
500:	learn: 4.2795525	test: 4.3153187	test1: 7.4011088	best: 7.4011088 (500)	total: 23.2s	remaining: 7m 20s
600:	learn: 3.8581968	test: 3.9114634	test1: 7.3752163	best: 7.3682036 (542)	total: 27.8s	remaining: 7m 15s
700:	learn: 3.4746593	test: 3.5489638	test1: 7.3751943	best: 7.3682036 (542)	total: 32.4s	remaining: 7m 10s
800:	learn: 3.1151109	test: 3.2038868	test1: 7.3857534	best: 7.3649586 (705)	total: 37.1s	remaining: 7m 5s
900:	learn: 2.8079867	test: 2.

err:  10.639382010225278
0:	learn: 18.9782837	test: 18.9597799	test1: 18.2983432	best: 18.2983432 (0)	total: 54.1ms	remaining: 9m 1s
100:	learn: 7.2989642	test: 7.2807054	test1: 8.8236005	best: 8.8236005 (100)	total: 4.66s	remaining: 7m 36s
200:	learn: 6.2349194	test: 6.2214639	test1: 8.5685924	best: 8.5461445 (195)	total: 9.26s	remaining: 7m 31s
300:	learn: 5.4726355	test: 5.4844551	test1: 8.4591519	best: 8.4482542 (292)	total: 13.8s	remaining: 7m 24s
400:	learn: 4.8122969	test: 4.8448633	test1: 8.3502418	best: 8.3502418 (400)	total: 18.4s	remaining: 7m 21s
500:	learn: 4.2556210	test: 4.3027142	test1: 8.3577161	best: 8.3028441 (471)	total: 23.1s	remaining: 7m 17s
600:	learn: 3.8111005	test: 3.8733223	test1: 8.3195269	best: 8.3028441 (471)	total: 27.8s	remaining: 7m 14s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 8.302844132
bestIteration = 471

Shrink model to first 472 iterations.


err:  13.689621561095098
0:	learn: 18.9500989	test: 18.9314435	test1: 18.9708408	best: 18.9708408 (0)	total: 54.3ms	remaining: 9m 3s
100:	learn: 7.3446104	test: 7.3311975	test1: 7.6310475	best: 7.6310475 (100)	total: 4.72s	remaining: 7m 42s
200:	learn: 6.2876865	test: 6.2721236	test1: 7.3190094	best: 7.3190094 (200)	total: 9.32s	remaining: 7m 34s
300:	learn: 5.5006384	test: 5.4999901	test1: 7.2741811	best: 7.2468075 (285)	total: 13.9s	remaining: 7m 26s
400:	learn: 4.8616689	test: 4.8766533	test1: 7.2206444	best: 7.2060009 (393)	total: 18.5s	remaining: 7m 22s
500:	learn: 4.3214565	test: 4.3639718	test1: 7.2349394	best: 7.1982161 (468)	total: 23.1s	remaining: 7m 18s
600:	learn: 3.8283236	test: 3.8892917	test1: 7.2007029	best: 7.1957363 (588)	total: 27.8s	remaining: 7m 14s
700:	learn: 3.4380624	test: 3.5158203	test1: 7.1855099	best: 7.1721853 (693)	total: 32.4s	remaining: 7m 10s
800:	learn: 3.0921349	test: 3.1849906	test1: 7.2260783	best: 7.1721853 (693)	total: 37.1s	remaining: 7m 5s
Stop

err:  10.852373112462672
0:	learn: 18.8847126	test: 18.8835382	test1: 19.2982969	best: 19.2982969 (0)	total: 51.9ms	remaining: 8m 38s
100:	learn: 7.4556155	test: 7.4450812	test1: 8.1505952	best: 8.1430294 (99)	total: 4.6s	remaining: 7m 30s
200:	learn: 6.3620710	test: 6.3616738	test1: 7.7678919	best: 7.7660394 (199)	total: 9.15s	remaining: 7m 26s
300:	learn: 5.5159292	test: 5.5417452	test1: 7.5756211	best: 7.5699142 (297)	total: 13.7s	remaining: 7m 22s
400:	learn: 4.8846164	test: 4.9257315	test1: 7.4065886	best: 7.3963461 (394)	total: 18.3s	remaining: 7m 18s
500:	learn: 4.3330410	test: 4.3914741	test1: 7.3458365	best: 7.3458365 (500)	total: 22.9s	remaining: 7m 14s
600:	learn: 3.8846901	test: 3.9600633	test1: 7.3818474	best: 7.3304103 (507)	total: 27.6s	remaining: 7m 11s
700:	learn: 3.4826321	test: 3.5723821	test1: 7.3511552	best: 7.3304103 (507)	total: 32.2s	remaining: 7m 6s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 7.330410316
bestIteration = 507

Shrink model 

err:  10.600364482928537
0:	learn: 18.8872859	test: 18.8862358	test1: 18.6267717	best: 18.6267717 (0)	total: 52.7ms	remaining: 8m 46s
100:	learn: 7.3949548	test: 7.3888193	test1: 8.1825582	best: 8.1793773 (97)	total: 4.7s	remaining: 7m 40s
200:	learn: 6.3228759	test: 6.3264785	test1: 7.9059035	best: 7.8936461 (197)	total: 9.31s	remaining: 7m 33s
300:	learn: 5.5072228	test: 5.5180770	test1: 7.6900740	best: 7.6892534 (290)	total: 13.9s	remaining: 7m 27s
400:	learn: 4.8527967	test: 4.8741328	test1: 7.6045843	best: 7.5759907 (391)	total: 18.5s	remaining: 7m 22s
500:	learn: 4.2916342	test: 4.3281598	test1: 7.5801635	best: 7.5759907 (391)	total: 23.1s	remaining: 7m 18s
600:	learn: 3.7920970	test: 3.8384131	test1: 7.5093533	best: 7.5093533 (600)	total: 27.8s	remaining: 7m 14s
700:	learn: 3.3941455	test: 3.4634287	test1: 7.5023151	best: 7.4913847 (627)	total: 32.4s	remaining: 7m 10s
800:	learn: 3.0343864	test: 3.1238110	test1: 7.5485112	best: 7.4913847 (627)	total: 37.7s	remaining: 7m 12s
Stop

...

err:  11.425919153850762
0:	learn: 18.9219170	test: 18.9197851	test1: 19.6363573	best: 19.6363573 (0)	total: 53.4ms	remaining: 8m 54s
100:	learn: 7.4404453	test: 7.4381126	test1: 8.9059738	best: 8.9059738 (100)	total: 4.69s	remaining: 7m 40s
200:	learn: 6.2745046	test: 6.2798266	test1: 8.5156488	best: 8.5156488 (200)	total: 9.32s	remaining: 7m 34s
300:	learn: 5.4948248	test: 5.5290798	test1: 8.2903782	best: 8.2894824 (299)	total: 13.9s	remaining: 7m 27s
400:	learn: 4.8687577	test: 4.9198935	test1: 8.1069774	best: 8.1024431 (398)	total: 18.6s	remaining: 7m 24s
500:	learn: 4.3076687	test: 4.3896234	test1: 8.0688686	best: 8.0614982 (452)	total: 23.2s	remaining: 7m 20s
600:	learn: 3.8102602	test: 3.9150271	test1: 8.0951051	best: 8.0614982 (452)	total: 27.9s	remaining: 7m 15s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 8.06149817
bestIteration = 452

Shrink model to first 453 iterations.


err:  11.842532914623352
0:	learn: 18.9344956	test: 18.9326684	test1: 19.4456434	best: 19.4456434 (0)	total: 55.8ms	remaining: 9m 17s
100:	learn: 7.4497745	test: 7.4482660	test1: 8.1104051	best: 8.1104051 (100)	total: 4.68s	remaining: 7m 38s
200:	learn: 6.3810908	test: 6.3874612	test1: 7.8507464	best: 7.8495847 (199)	total: 9.23s	remaining: 7m 29s
300:	learn: 5.5196806	test: 5.5367227	test1: 7.5966400	best: 7.5966400 (300)	total: 13.8s	remaining: 7m 26s
400:	learn: 4.8544471	test: 4.8893533	test1: 7.6220478	best: 7.5488674 (335)	total: 18.4s	remaining: 7m 21s
500:	learn: 4.2917280	test: 4.3451323	test1: 7.5863159	best: 7.5488674 (335)	total: 23s	remaining: 7m 16s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 7.548867355
bestIteration = 335

Shrink model to first 336 iterations.


err:  10.839257827397718
0:	learn: 18.8773540	test: 18.8759242	test1: 18.8304280	best: 18.8304280 (0)	total: 53.5ms	remaining: 8m 55s
100:	learn: 7.3141357	test: 7.2946595	test1: 8.4758994	best: 8.4758994 (100)	total: 4.71s	remaining: 7m 41s
200:	learn: 6.2989518	test: 6.2834939	test1: 8.3132910	best: 8.3132910 (200)	total: 9.29s	remaining: 7m 32s
300:	learn: 5.4072663	test: 5.4112144	test1: 8.1317764	best: 8.1262453 (296)	total: 13.9s	remaining: 7m 28s
400:	learn: 4.7581312	test: 4.7898979	test1: 8.0231873	best: 8.0200149 (399)	total: 18.5s	remaining: 7m 23s
500:	learn: 4.2160926	test: 4.2671062	test1: 8.0077385	best: 7.9811478 (464)	total: 23.2s	remaining: 7m 19s
600:	learn: 3.7674735	test: 3.8340969	test1: 7.9132723	best: 7.9089537 (599)	total: 27.8s	remaining: 7m 15s
700:	learn: 3.3635693	test: 3.4470028	test1: 7.8646388	best: 7.8600763 (687)	total: 32.5s	remaining: 7m 10s
800:	learn: 2.9975401	test: 3.1028465	test1: 7.8486230	best: 7.8356752 (787)	total: 37s	remaining: 7m 5s
900:	

err:  11.063317838755115


12.371004528563706

In [ ]:
np.mean(errcb2)
d = {'ID': test_id, 'pm2_5': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'pm2_5']]
sub.to_csv("catboost_sub.csv",index  = False)
sub.pm2_5.min(),sub.pm2_5.max()
sub.head()

12.371004528563706

(25.334528787108827, 87.3422465271084)

ID      pm2_5
0  ID_UOH62J0XHX  41.295332
1  ID_4OPWSB0UHJ  67.046616
2  ID_3SLMNNG1Z3  42.223459
3  ID_MFUHTXYPM4  60.363134
4  ID_TY1DAND8ZP  57.382530

In [ ]:
sub.to_csv("catboost_sub.csv",index  = False)
from google.colab import files
files.download("catboost_sub.csv")    # 15.2133 on LB